This notebook will be fully reproducible (no changes needed) as long as you setup an ngrok authtoken at https://ngrok.com/. Details are at step 3.

**Step 0 - Data Collection**

We could get economic data for this App from the Federal Reserve Economic Database (FRED to its friends) using the pandas datareader library. However I've chosen to take it directly from the US Bureau of Labor Statistics website as you don't need to know all the different codes for industries and data types. Then I've removed certain types of data from the dataframe for our App.

In [ ]:
import pandas as pd

#This dataframe contains the JOLTS estimates that we'll be using for our app
base_url = 'https://download.bls.gov/pub/time.series/jt/' 
data_df = pd.read_csv(base_url + 'jt.data.1.AllItems', sep='\t')

#The other dataframes below are lookup tables that will be used later on
dataelement_df = pd.read_csv(base_url + 'jt.dataelement', sep='\t')
period_df = pd.read_csv(base_url + 'jt.period', sep='\t')
ratelevel_df = pd.read_csv(base_url + 'jt.ratelevel', sep='\t')
industry_df = pd.read_csv(base_url + 'jt.industry', sep='\t')
series_df = pd.read_csv(base_url + 'jt.series', sep='\t')
series_df = series_df.merge(industry_df, on='industry_code', how='left')
series_df = series_df.merge(ratelevel_df, on='ratelevel_code', how='left')
series_df = series_df.merge(dataelement_df, on='dataelement_code', how='left')

data_df = data_df.merge(series_df, on='series_id                     ', how='left')
data_df = data_df.merge(period_df, on='period', how='left')

# remove whitespace from some column names and strings in one column
data_df.columns = data_df.columns.str.replace(' ', '')
data_df['series_id'] = data_df['series_id'].str.strip()

data_df = data_df[(data_df.sizeclass_code == 0) & (data_df.state_code == '00') \
                  & (data_df.period != 'M13')]

final_df = data_df[['year', 'period_name', 'series_id', 'industry_text', 'seasonal',\
                    'ratelevel_text', 'dataelement_text', 'value']]
indList = final_df.industry_text.unique()
deList = final_df.dataelement_text.unique()
seriesList = final_df.series_id.unique()
colorList = 10 * ['#F0F8FF', '#ADff2F', '#4B0082', '#B22222', '#E0FFFF', '#32CD32', \
                  '#CD853F', 'FF6347', '#4682B4', '#98FB98']
    

**Step 1 - Setup directories for Flask**

In [ ]:
import os

if not os.path.exists('templates'):
  os.makedirs('templates')
if not os.path.exists('static'):
  os.makedirs('static')


**Step 2 - Writing HTML files to your templates directory**

Now we're getting to the core of what we're doing here. We'll do this in two parts. First, we're creating a header that will be on each page our App loads. Second we'll write individual pages to the templates directory that contain graphs and tables for each industry-datatype combination.


In [ ]:
header = '''
<!DOCTYPE html>
<html lang="en">
<head>
  <title>JOLTS Chart App</title>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <script src="https://cdnjs.cloudflare.com/ajax/libs/Chart.js/3.9.1/chart.min.js"></script>
  <script src="https://maxcdn.bootstrapcdn.com/bootstrap/4.4.1/js/bootstrap.min.js"></script>
  <script src="https://cdnjs.cloudflare.com/ajax/libs/jquery/3.6.1/jquery.min.js"></script>
  <script src="https://unpkg.com/gridjs/dist/gridjs.umd.js"></script>
  <script src="https://cdnjs.cloudflare.com/ajax/libs/gridjs/5.1.0/gridjs.production.min.js"></script>
  <link rel="stylesheet" href="https://unpkg.com/gridjs/dist/theme/mermaid.min.css" type="text/css" />
  <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css">

<div class="container text-center">
  <h5> US Job Openings and Labor Turnover Survey Application</h5>
</div>

<nav class="navbar navbar-expand-lg navbar-light bg-light">
  <a class="navbar-brand" href="#">Home</a>

  <div class="collapse navbar-collapse">
    <form action = "/JOLTSReview" method = "post">
      <div class="row">
        <div class="col">
          <div class="form-group">
            <label for="etSelect">Estimation Type</label>
            <select class="form-control" id="etSelect" name='et'>
              <option>NSA</option>
              <option>SA</option>
            </select>
          </div>
        </div>
        <div class="col">
          <div class="form-group">
            <label for="dtSelect">Data Element</label>
            <select class="form-control" id="dtSelect" name='dt'>
              {% for deItem in deList %}
                 <option>{{ deItem }}</option>
              {% endfor %}
            </select>
          </div>
        </div>
        <div class="col">
          <div class="form-group">
            <label for="catSelect">Category</label>
            <select class="form-control" id="catSelect" name='cat'>
              <option>Rates</option>
              <option>Levels</option>
            </select>
          </div>
        </div>
        <div class="col">
          <div class="form-group">
            <label for="indSelect">Industry / Region</label>
            <select class="form-control" id="indSelect" name='ind'>
              {% for indItem in indList %}
                 <option>{{ indItem }}</option>
              {% endfor %}
            </select>
          </div>
        </div>
        <div class="col">
        <button type="submit" class="button button0">Load Data</button>
		    </div>
      </div>
    </form>
  </div>
</nav>
</head>
'''
file = open("templates/header.html","w")
file.write(header)
file.close()

In [ ]:
def write_html(seriesID):
    # writes all the pages that can be accessed by a user
    htmlFile = 'templates/%s.html' % (seriesID)
    df_selected = final_df[final_df['series_id'] == seriesID]
    industry_text = df_selected['industry_text'].iloc[0]
    dataelement_text = df_selected['dataelement_text'].iloc[0]
    if seriesID[0:3] == 'JTU':
        nsa_sa = 'NSA'
    else:
        nsa_sa = 'SA'        
    yearList = df_selected.year.unique()[:-1]
    estList = df_selected['value'].to_list()
    topArgs = {'topArg0': nsa_sa, 'topArg1': dataelement_text, 'topArg2': industry_text}

    part0 = '''
    {{% include 'header.html' %}}
      <body>
      <h6 align='center'>{topArg0} {topArg1} - {topArg2}</h6>
        <div class='mx-auto' style='width: 1000px; height: 500px;'>
           <canvas id='JOLTSChart'></canvas>
        </div>
    <script type='text/javascript'>
    // labels along the x-axis
    const months = ['Dec(PY)','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'];
    // For drawing the lines
    '''.format(**topArgs)
    html_part0 = open(htmlFile,"w")
    html_part0.write(part0)
    html_part0.close()
    
    html_part1 = open(htmlFile, "a+")
    for yearID, year in enumerate(yearList):
        year = yearID + 2001
        estListStart = (yearID*12)
        estListEnd = estListStart + 13
        html_part1.writelines("let array%s" % (year) +" = " + "%s;" %(estList[estListStart:estListEnd]) + "\n")
    html_part1.close()

    part2 = '''
    var ctx = document.getElementById('JOLTSChart');
    var JOLTSChart = new Chart(ctx, {
    type: 'line',
    data: {
    labels: months,
    datasets: [
    '''
    html_part2 = open(htmlFile,"a+")
    html_part2.write(part2)
    html_part2.close()

    html_part3 = open(htmlFile, "a+")
    for yearID, year in enumerate(yearList):
        html_part3.writelines("{" + "\n")
        html_part3.writelines("   data: array%s" % (year + 1) + "," + "\n")
        html_part3.writelines("   type: 'line'" + ","  + "\n")
        html_part3.writelines("   label: '%d'" % (year + 1) + "," + "\n")
        html_part3.writelines("   borderColor: '%s'" % (colorList[yearID]) + "," + "\n")
        html_part3.writelines("   fill: false" + "," + "\n")
        html_part3.writelines("   lineTension: 0" + "," + "\n")
        if yearID > len(yearList) - 10:
            html_part3.writelines("   hidden: false" + "\n")
        else:
            html_part3.writelines("   hidden: true" + "\n")
        html_part3.writelines(" }" + "," + "\n")
    html_part3.close()
    
    part4 = '''
    ]
    }
    });
    </script>
    <br>
    <center>
    <div id='JOLTSTable'></div>
    </center>
    <script>
    data = [
    '''
    html_part4 = open(htmlFile,"a+")
    html_part4.write(part4)
    html_part4.close()

    html_part5 = open(htmlFile, "a+")
    for yearID, year in enumerate(yearList):
        html_part5.writelines("[%s].concat(array%s)" % (year + 1, year + 1) + "," + "\n")
    html_part5.close()

    part6 = '''
    ];
    new gridjs.Grid({
    columns: ['Year'].concat(months),
    data: data,
    colWidths: [150,100,100,100,100,100,100,100,100,100,100,100,100,100],
    pagination: true,
    style: {
      table: {
        border: '3px solid #ccc'
    },
    th: {
      'background-color': 'rgba(0, 0, 0, 0.1)',
      color: '#000',
      'border-bottom': '3px solid #ccc',
      'text-align': 'center'
    },
    td: {
      'text-align': 'center'
    }
  }
    }).render(document.getElementById("JOLTSTable"));
    
        </script>
      </body>
    </html>
    '''
    html_part6 = open(htmlFile,"a+")
    html_part6.write(part6)
    html_part6.close()

for series in seriesList:
    write_html(seriesID=series)


**Step 3 - Ngrok setup**


In [ ]:
!pip install flask-ngrok --quiet
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
!tar -xvf /content/ngrok-stable-linux-amd64.tgz

# paste your own AuthToken here (after signing up for free at ngrok website) and execute this command
!./ngrok authtoken 3X62gnpfg01wmWLQi24h5AB_4akYioc8NALGzz3KeBRsn

--2022-09-21 15:15:59--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 18.205.222.128, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13770165 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.tgz’

ngrok-stable-linux- 100%[===================>]  13.13M  53.9MB/s    in 0.2s    

2022-09-21 15:15:59 (53.9 MB/s) - ‘ngrok-stable-linux-amd64.tgz’ saved [13770165/13770165]

ngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


**Step 4 - Flask setup and deploying web application to localhost and NGROK**

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, request
import pandas as pd

app = Flask(__name__)
run_with_ngrok(app)

@app.route("/", methods=['GET','POST'])
def start():
    return render_template("header.html", deList=deList, indList=indList)

@app.route("/JOLTSReview", methods=['GET','POST'])
def JOLTSReview():
    # This funtion is taking user inputs and directing to desired chart/table template
    et = request.form.get('et')
    dt = request.form.get('dt')
    cat = request.form.get('cat')
    ind = request.form.get('ind')
    if et != 'SA':
        etSelection = 'JTU'
    else:
        etSelection = 'JTS'
    indSelection = industry_df['industry_code'].astype(str).str.zfill(6).loc[industry_df['industry_text'].str.contains(ind)].to_string(index=False)
    stateSelection = '00'
    areaSelection = '00000'
    sizeSelection = '00'
    dtSelection = dataelement_df['dataelement_code'].loc[dataelement_df['dataelement_text'].str.contains(dt)].to_string(index=False)
    catSelection = cat[:1].upper()
    seriesSelection = etSelection + indSelection + stateSelection + areaSelection + sizeSelection + dtSelection + catSelection
    return render_template("%s.html" % (seriesSelection), industry_text=ind, dataelement_text=dt, nsa_sa=et, \
                           datatype_text=cat, deList=deList, indList=indList)
    
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://26e9-34-73-53-21.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [21/Sep/2022 15:38:26] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Sep/2022 15:38:28] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [21/Sep/2022 15:38:36] "POST /JOLTSReview HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Sep/2022 15:39:12] "POST /JOLTSReview HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Sep/2022 15:39:28] "POST /JOLTSReview HTTP/1.1" 200 -
